In [1]:
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# adjust this to your clone location
DATA_DIR = "helsinki-dataset"
EDFs     = sorted([f for f in os.listdir(DATA_DIR) if f.endswith(".edf")])
CSVs     = {
    "A": os.path.join(DATA_DIR, "annotations_2017_A_fixed.csv"),
    "B": os.path.join(DATA_DIR, "annotations_2017_B.csv"),
    "C": os.path.join(DATA_DIR, "annotations_2017_C.csv"),
}


In [2]:
dfA = pd.read_csv(CSVs["A"])
dfA.head()

,1,2,3,4,5,6,7,8,9,10,...,70,71,72,73,74,75,76,77,78,79
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
